# Thermoelasticity-based fatigue life identification

## Install package via pip

In [ ]:
pip install IRFlife

## Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pysfmov as sfmov
import IRFLife

## Material parameters

In [2]:
k = 6.51               # Slope endurance curve
B = 800.26             # Endurance curve limit [MPa]
C = 7.94 * 10**18      # Fatigue strenght [MPa**k]
km = 1.2 * 10**(-8)    # Thermoelastic coefficient [Pa**(-1)]

## Input thermal video

In [3]:
filename = './data/rec.sfmov'        # Filename of thermal acquisition
data = sfmov.get_data(filename)      # Using pysfmov to open it as numpy array [°C]

fs = 400                             # Smapling frequency [Hz]
dt = 1 / fs                          # Time step [s]

stress = 10 * (data / km ) * 10**-6  # Stress [MPa]

## Class initialization

In [4]:
td =  IRFLife(stress, dt)        # Class initialization

### Natural frequency identification from thermal video

#### If the location is picked with the mouse click

In [5]:
%matplotlib qt               
td.loc_selection()           # Mouse selection of central pixel of the roi

In [6]:
f = td.nf_identification()   # Natural frequency identification

#### If the location is settled with roi coordinates

In [ ]:
band_pass = [5,100]         # Band pass filter applied during the natural frequency identification
roi_size = 5                # ROI size [pixel]
location = (39, 79, 5, 5)   # Location of interest in the field of view [pixel]

f = td.nf_identification(location = location, roi_size = roi_size, band_pass = band_pass)

## Fatigue life estimation

#### If the fatigue life is wanted at a particular location

In [7]:
location = (39, 79, 5, 5)   # Location of interest in the field of view [pixel]

In [10]:
md = td.get_life(C, k, 'Modal', f = f, location = location)
tb = td.get_life(C, k, 'TovoBenasciutti', location = location)
dk = td.get_life(C, k, 'Dirlik', location = location)
rf = td.get_life(C, k, 'Rainflow', location = location)

100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 781.98it/s]


In [11]:
print(f'          Rainflow: {rf:4.0f} s')
print(f'            Dirlik: {dk:4.0f} s')
print(f'  Tovo-Benasciutti: {tb:4.0f} s')
print(f'             Modal: {md:4.0f} s')

          Rainflow: 44125 s
            Dirlik: 33250 s
  Tovo-Benasciutti: 35668 s
             Modal: 8279399575 s


#### If the fatigue life is wanted even in the spatial domain

In [12]:
md = td.get_life(C, k, 'Modal', f = f)
tb = td.get_life(C, k, 'TovoBenasciutti')
dk = td.get_life(C, k, 'Dirlik')
rf = td.get_life(C, k, 'Rainflow')

100%|██████████████████████████████████████████████████████████████████████████| 20480/20480 [00:18<00:00, 1119.10it/s]


In [13]:
plt.figure()

plt.subplot(2,2,1)
plt.imshow(dk)
plt.colorbar()
plt.title('Dirlik')

plt.subplot(2,2,2)
plt.imshow(md)
plt.colorbar()
plt.title('Modal approach')

plt.subplot(2,2,3)
plt.imshow(rf)
plt.colorbar()
plt.title('Rainflow')

plt.subplot(2,2,4)
plt.imshow(tb)
plt.colorbar()
plt.title('Tovo-Benasciutti')

Text(0.5, 1.0, 'Tovo-Benasciutti')